# Borrower reliability research

The customer is the credit department of the bank. It is necessary to find out whether the marital status and the number of children of the client affect the fact of repaying the loan on time. Input data from the bank are statistics of the solvency of customers.

The results of the study will be taken into account when building a **credit scoring** model that is a special system that evaluates the ability of a potential borrower to repay a loan to a bank.

**The purpose of the research** is answer four questions:
1. Is there a relationship between having children and repaying a loan on time?
2. Is there a relationship between marital status and loan repayment on time?
3. Is there a relationship between the level of income and the return of the loan on time?
4. How do different purposes of a loan affect its repayment on time?.

**Research progress**

Nothing is known about the quality of the data, so it is necessary to conduct a preliminary review, processing (checking the data for errors, missing values, duplicates) and correcting existing errors, missing values, duplicates.
It will also be necessary to categorize the data to answer the questions.
Thus, the study will take place in four steps:
1. Data preview.
2. Data preprocessing.
3. Data categorization.
4. Getting answers to questions.

## Step 1. Data preview

In [1]:
# import pandas, data file opening, saving in variable data and displayin the first ten rows
import pandas as pd
data = pd.read_csv('....csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# get general information about the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# count missing values
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64



**Conclusions:**

1. The table has 12 columns, the data type in some columns is different. Occurring data types in a table:
   - float64 - 2 columns
   - int64 - 5 columns
   - object - 5 columns
   
   According to the data documentation, the columns are characterized as follows:
   - children - amount of kids in the family (тип: int64)
   - days_employed - total work experience in days (тип: float64)
   - dob_years - client's age in years (тип: int64)
   - education - client's education level (тип: object)
   - education_id - educational level ID (тип: object),
   - family_status - famiy status (тип: object)
   - family_status_id - family status ID (тип: object),
   - gender - client's gender (тип: object)
   - income_type - income type (тип: object)
   - debt - loan repayment (тип: int)
   - total_income - monthly income (тип: float)
   - purpose - purpose of the loan (тип: object)
2. Our conclusions:
   - column names are specified without errors in the so-called "snake case";
   - general information about the data shows that there are missing values in the data in the days_employed and total_income columns;
   - in the days_employed column, the data is negative numbers, we need to change their values to positive ones, because as a result of loading data, errors could occur that led to incorrect data. Also, the type of this column needs to be changed from real to integer and missing values must be corrected;
   - in the education column, all data must be converted to lowercase;
   - in the gender column, convert all values to lowercase;
   - in the total_income column, we must change the type of all values to integer, in addition, the missing values must be processed.
3. Explicit duplicates have also been identified that will need to be processed.

Let's consider each column in more detail to identify anomalies.

In [4]:
# display general information
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [5]:
# search for anomalies in the children column
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In the above data output, you can see that there are 2 anomalies in the children column:

1. the number of children is 20 for 76 clients, it certainly happens that a family has many children, but the fact that there are 76 such clients is alarming, most likely there is an error in the data, a human factor.
2. There is another anomaly: this is the number of children -1 in 47 clients. In my opinion, there is also a human factor here.

These data can be corrected, for example -1, replaced by one, and with those who have 20 children, do the following: determine the age category of such people and take the median value in the sample for the specified age.

In [6]:
# let's take a closer look at the children column, let's see which clients have 20 children
data[data['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


As expected, this is an error in the data, because the age of clients with 20 children is 21 years old.

In order to correct incorrect values, we will apply the following operations: -1 will be replaced by 1, and the values ​​of 20 will be replaced by the median for each age category. Such a replacement is justified, since the number of elements with an error is 0.35%, therefore, it cannot significantly affect the result of the study..

In [7]:
# change the values -1 to 1 in the children column, display information on the children column
# to make sure that the changes have occurred
data.loc[data['children'] == -1, 'children'] = 1
print(data['children'].value_counts())

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


For a more correct replacement of the error in the data, we determine the median by the number of children depending on the age category

In [8]:
# calculate the median for the children column depending on the age of the clients
median_children_before_25 = int(data.loc[data['dob_years'] <= 25]['children'].median())
median_children_26_30 = int(data.loc[(data['dob_years'] >= 26)&(data['dob_years'] <= 30)]['children'].median())
median_children_31_35 = int(data.loc[(data['dob_years'] >= 31)&(data['dob_years'] <= 35)]['children'].median())
median_children_36_40 = int(data.loc[(data['dob_years'] >= 36)&(data['dob_years'] <= 40)]['children'].median())
median_children_41_45 = int(data.loc[(data['dob_years'] >= 41)&(data['dob_years'] <= 45)]['children'].median())
median_children_46_50 = int(data.loc[(data['dob_years'] >= 46)&(data['dob_years'] <= 50)]['children'].median())
median_children_51_55 = int(data.loc[(data['dob_years'] >= 51)&(data['dob_years'] <= 55)]['children'].median())
median_children_upper_56 = int(data.loc[data['dob_years'] >= 56]['children'].median())

In [9]:
print('Медиана для возраста до 25 лет:', median_children_before_25)
print('Медиана для возраста от 26 до 30 лет:', median_children_26_30)
print('Медиана для возраста от 31 до 35 лет:', median_children_31_35)
print('Медиана для возраста от 36 до 40 лет:', median_children_36_40)
print('Медиана для возрастаот 41 до 45 лет:', median_children_41_45)
print('Медиана для возраста от 46 до 50 лет:', median_children_46_50)
print('Медиана для возраста от 51 до 55 лет:', median_children_51_55)
print('Медиана для возраста старше 60 лет:', median_children_upper_56)

Медиана для возраста до 25 лет: 0
Медиана для возраста от 26 до 30 лет: 0
Медиана для возраста от 31 до 35 лет: 1
Медиана для возраста от 36 до 40 лет: 1
Медиана для возрастаот 41 до 45 лет: 0
Медиана для возраста от 46 до 50 лет: 0
Медиана для возраста от 51 до 55 лет: 0
Медиана для возраста старше 60 лет: 0


The difference in values is small, therefore, for a more correct replacement, we will use the value of the median equal to 0, but for the age period: from 31 to 40 years, fill in the incorrect data with a median value of 1.

In [10]:
# replacing values in column children == 20 with medians
data.loc[(data['children'] == 20)&(data['dob_years'] >= 31)&(data['dob_years'] <= 40), 'children'] = median_children_31_35
data.loc[data['children'] == 20, 'children'] = median_children_before_25
print(data['children'].value_counts())

0    14202
1     4888
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


In the final call, you can see that all the anomalous values ​​in the column have been replaced and thus all clients can be assigned to six categories:
- 0 - no child
- 1 - one child
- 2 - two children
- 3 - three children
- 4 - four children
- 5 - five children

In [11]:
# replace the negative values ​​in the days_employed column with positive ones
data.loc[data['days_employed'] < 0, 'days_employed'] = 0 - data['days_employed']
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


As you can see in this column, all negative numbers have been replaced with positive ones. Below we will check to make sure that there are no negative data.

In [12]:
# check for negative data
print(data[data['days_employed'] < 0])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


In [13]:
# analyze the dob_years column
print(data['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


There is one anomaly visible in the above data:
- 101 people have age 0 years. Perhaps this error was due to the fact that this information was not specified by customers. Using the table below, we can try to restore the approximate range of ages in the indicated lines. Let's take the median age of people in the indicated column and replace the anomalous values. The share of this anomaly in the total amount of data is 0.47%, so such a replacement should not significantly affect the results of the study..

In [14]:
# calculate the median over the dob_years column and store this value in the dob_years_median variable as an integer
dob_years_median = int(data['dob_years'].median())
print('Медианное значение возраста:', dob_years_median)

Медианное значение возраста: 42


In [15]:
# replace the 0 values in the dob_years column with dob_years_median and display the unique values of the column on the screen
data.loc[data['dob_years'] == 0, 'dob_years'] = dob_years_median
print(data['dob_years'].value_counts())

42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


In the above output, we can see that there are no more customers with an age value of 0.

In [16]:
# analyze the client's education level
print(data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


In the data above, the following levels of education are visible:
1. average
2. unfinished higher
3. higher
4. initial
5. degree

It is difficult to identify the total number of clients for each level, since the data are presented in different registers. Let's change this data and compare the resulting column with the education_id column. This is necessary in order to understand whether the category from 0 to 4 is correctly indicated for each level of education

In [17]:
# converting all values in the education column to lowercase
data['education'] = data['education'].str.lower()

In [18]:
# display the values ​​of the education and education_id columns
print('Значения столбца education:')
print(data['education'].value_counts())
print()
print('Значения столбца education_id:')
print(data['education_id'].value_counts())

Значения столбца education:
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Значения столбца education_id:
1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


As a result of the above actions, the readability of the education column has been increased. We can also conclude that the categories from the education_id column are correct, and now we can see for which level of education which category is indicated:

- 0 - высшее
- 1 - среднее
- 2 - неоконченное высшее
- 3 - начальное
- 4 - ученая степень

But in the future, we would like information about the correspondence of the level of education to its category to be indicated in the documentation for the data.

In [19]:
# analyze family_status column
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


The above data shows the following types of marital status:

- женат/замужем
- гражданский брак
- не женат/не замужем
- в разводе
- вдовец/вдова

The resulting table shows that the data are presented in different registers. Let's change this data and compare the resulting column with the family_status_id column. This is necessary in order to understand whether the category from 0 to 4 is correctly indicated for each type of marital status.

In [20]:
# приведение всех значений в столбце family_status к нижнему регистру
data['family_status'] = data['family_status'].str.lower()

In [21]:
# выведем на экран значения столбца education и education_id
print('Значения столбца family_status:')
print(data['family_status'].value_counts())
print()
print('Значения столбца family_status_id:')
print(data['family_status_id'].value_counts())

Значения столбца family_status:
женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Значения столбца family_status_id:
0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


As a result, we can conclude that the categories from the family_status_id column are correct, and now you can see for which type of marital status which category is indicated:

- 0 - married
- 1 - civil marriage
- 2 - widower/widow
- 3 - divorced
- 4 - not married

But in the future, we would like information about the correspondence of the marital status and its category to be indicated in the documentation for the data.

In [22]:
# analyze the gender column
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


This column showed that one of the values indicated the wrong gender, most likely this is a random error. Also, in this column, all values are in upper case, and it would be more convenient for analysis if all values of the column were in lower case.
We will not correct the value in the column equal to XNA, since it is impossible to accurately determine the gender of a person and correct this value. Therefore, we will correct only the case, we will bring all the values to lower case.

In [23]:
# converting all values in the gender column to lowercase
data['gender'] = data['gender'].str.lower()
print(data['gender'].value_counts())

f      14236
m       7288
xna        1
Name: gender, dtype: int64


We see that the data in the column has changed. In the current data analysis, this column will not be useful to us. But for the future, it is desirable to clarify the data for the client who incorrectly indicated the gender.

In [24]:
# analyze income_type column
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64


No anomalies or data errors were found in this column.

In [25]:
# analyze debt column
print(data['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64


No anomalies were found in this column either. The only thing the data documentation doesn't say is what 0 and 1 means, is it the number of debts, or is it a boolean expression False for 0 and True for 1.
Since no more data on the number of debts is indicated and we only have values of 0 and 1, we will assume that these are boolean values, for which:

- 0 - False
- 1 - True

But in the future, this column should have a comment in the data documentation.

In [26]:
# Let's analyze the total_income column. We will not be able to find unique values for this column, as there will be many records
# but we can see, for example, the minimum and maximum values, in order to exclude the presence of negative numbers in the column
print('Максимальное значение с столбце total_income: ', data['total_income'].max())
print('Минимальное значение с столбце total_income: ', data['total_income'].min())

Максимальное значение с столбце total_income:  2265604.028722744
Минимальное значение с столбце total_income:  20667.26379327158


According to the analysis, it can be seen that there are no negative values, respectively, in this column, we need to fill in the missing values and change the data type from real to integer.

In [27]:
# analzye purpose column
print(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

According to the data above, it can be seen that there are many similar purposes for obtaining a loan in the column, but it is very difficult to identify them, since the data differ in declensions, the same operation is described in different words. In order to bring these values to a single form, it is necessary to lemmatize and group the same goals based on the lemmas

In [28]:
# output the final dataset after all transformations
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу


The final table shows that all errors in the values (different registers, negative numbers, anomalous values) have been eliminated.

It remains to carry out the final processing of the data: correct the missing values, change the data type in the columns from real to integer, remove duplicates, lemmatize the data in the last column to highlight categories of loan purposes.

## Step 2. Data preprocessing

### Missing values preprocessing

In order to process the missing values, let's look at the table again and determine in which columns there is not enough data.

In [29]:
# find all rows with missing values
display(data[data['days_employed'].isna()])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,m,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,m,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,не женат / не замужем,4,f,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,f,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,f,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,среднее,1,женат / замужем,0,m,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,f,сотрудник,0,NaN,свадьба
21497,0,NaN,48,высшее,0,женат / замужем,0,f,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,f,сотрудник,0,NaN,строительство жилой недвижимости


The resulting table shows that the missing values in the days_employed and total_income columns are the same. This suggests that customers most likely did not provide this data themselves. Let's fill in the missing values using grouping by features and the median in these groups by the days_employed and total_income columns.

But before starting to fill in the missing values in the values, additional processing of the days_employed column is necessary.

Let's display this table again.

In [30]:
# display the sorted values of the days_employed column
display(data[data['days_employed'] > 16920])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,f,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,не женат / не замужем,4,f,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,m,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,f,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,m,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,m,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,m,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,среднее,1,женат / замужем,0,f,пенсионер,0,153864.650328,сделка с автомобилем


In the resulting output, we saw the following anomaly, for example, take row 18: the length of work in it is 400281 days, if we translate this number into years, based on the fact that when forming the length of work, the year is taken equal to 360 days, we get that a person before leaving retired worked: 400281 / 360 = 1111 years! This cannot be true.

The value taken in the formula: 16920 is formed as follows. Let's assume that a person worked from 18 to 65 years old, his total length of work in years is 47 years (65 - 18). To represent this value in days, multiply the number of years worked by 360. We get: 47 * 360 = 16920.

And there are 3447 of 21525 such rows in the table, which is about 16% of all data. Perhaps such distortions were obtained by incorrect recalculation of the data. This is a fairly large value, so if we consider the median value even by categories, we will get significant distortions in the data.

Let's try to transform these anomalies. Find out for people of what age such an anomaly is inherent.

In [31]:
# output the number of unique ages that have the specified anomaly
print(data[data['days_employed'] > 16920]['dob_years'].value_counts())

59    254
60    243
62    235
61    216
57    212
58    208
63    192
56    184
64    179
55    162
54    145
66    139
65    136
67    132
53    105
52     95
68     80
69     74
51     73
50     61
70     54
71     48
49     30
72     28
42     26
48     20
46     13
47     13
45     11
44     10
43      9
38      8
40      7
73      6
41      6
37      5
74      4
39      4
27      3
36      3
34      3
32      3
33      2
26      2
31      1
35      1
28      1
22      1
Name: dob_years, dtype: int64


According to the obtained conclusion, it can be seen that this anomaly occurs in all age categories. For a more correct calculation and filling of missing values in the table, we will correct the anomalies. Let's rewrite the data of this column, for this we take the median for each age category. Age categories will be done in increments of 5, the first age category is up to 25 years old inclusive, and the last one is over 71 years old inclusive.

In [32]:
# write a function that determines the median knowledge for a given interval
def get_median_years_employed(first_volume, second_volume):
    '''
    Determines the median of values from a specified range of values
    
    param: first_volume - first value in range inclusive
    param: second_volume - second value in itnerval inclusive
    return: median of values ​​from the specified interval
    '''
    return data.loc[(data['dob_years'] >= first_volume)&(data['dob_years'] <= second_volume)]['days_employed'].median()

In [33]:
# calculation of medians of the first and last categories, as well as calculation of medians of intervals using the function
median_before_25 = data.loc[data['dob_years'] <= 25]['days_employed'].median()
median_26_30 = get_median_years_employed(26, 30)
median_31_35 = get_median_years_employed(31, 35)
median_36_40 = get_median_years_employed(36, 40)
median_41_45 = get_median_years_employed(41, 45)
median_46_50 = get_median_years_employed(46, 50)
median_51_55 = get_median_years_employed(51, 55)
median_56_60 = get_median_years_employed(56, 60)
median_61_65 = get_median_years_employed(61, 65)
median_66_70 = get_median_years_employed(66, 70)
median_upper_71 = data.loc[data['dob_years'] >= 71]['days_employed'].median()

In [34]:
# display the resulting median values
print('Медиана стажа работы в категории до 25 лет:', median_before_25)
print('Медиана стажа работы в категории от 26 до 30 лет:', median_26_30)
print('Медиана стажа работы в категории от 31 до 35 лет:', median_31_35)
print('Медиана стажа работы в категории от 36 до 40 лет:', median_36_40)
print('Медиана стажа работы в категории от 41 до 45 лет:', median_41_45)
print('Медиана стажа работы в категории от 46 до 50 лет:', median_46_50)
print('Медиана стажа работы в категории от 51 до 55 лет:', median_51_55)
print('Медиана стажа работы в категории от 56 до 60 лет:', median_56_60)
print('Медиана стажа работы в категории от 61 до 65 лет:', median_61_65)
print('Медиана стажа работы в категории от 66 до 70 лет:', median_66_70)
print('Медиана стажа работы в категории от 71 года:', median_upper_71)

Медиана стажа работы в категории до 25 лет: 797.5918331374241
Медиана стажа работы в категории от 26 до 30 лет: 1228.638681675905
Медиана стажа работы в категории от 31 до 35 лет: 1496.0708150968378
Медиана стажа работы в категории от 36 до 40 лет: 1812.084853902544
Медиана стажа работы в категории от 41 до 45 лет: 2029.7750653706182
Медиана стажа работы в категории от 46 до 50 лет: 2411.5253843148307
Медиана стажа работы в категории от 51 до 55 лет: 3682.216121806053
Медиана стажа работы в категории от 56 до 60 лет: 336682.4288389096
Медиана стажа работы в категории от 61 до 65 лет: 353424.71246236726
Медиана стажа работы в категории от 66 до 70 лет: 361108.6961660925
Медиана стажа работы в категории от 71 года: 360170.42288407945


One feature is visible in the above median values: starting from the category of 56 years and further, the median is from 336682 days, which indicates that in all values, starting from 56 years of age, there is an error in the dob_years column. For the specified intervals, we will not replace anomalous values with medians, since the data for these categories cannot be used. In addition, we need to understand why these categories have such errors. Perhaps this error is of some technical nature or wrong recalculation in the data.

In [35]:
# replace the anomalous values of the interval with the values of the median, for all intervals except those
# that start from 56 years
data.loc[(data['dob_years'] <= 25)&(data['days_employed'] > 16920), 'days_employed'] = median_before_25
data.loc[(data['dob_years'] >= 26)&(data['dob_years'] <= 30)&(data['days_employed'] > 16920), 'days_employed'] = median_26_30
data.loc[(data['dob_years'] >= 31)&(data['dob_years'] <= 35)&(data['days_employed'] > 16920), 'days_employed'] = median_31_35
data.loc[(data['dob_years'] >= 36)&(data['dob_years'] <= 40)&(data['days_employed'] > 16920), 'days_employed'] = median_36_40
data.loc[(data['dob_years'] >= 41)&(data['dob_years'] <= 45)&(data['days_employed'] > 16920), 'days_employed'] = median_41_45
data.loc[(data['dob_years'] >= 46)&(data['dob_years'] <= 50)&(data['days_employed'] > 16920), 'days_employed'] = median_46_50
data.loc[(data['dob_years'] >= 51)&(data['dob_years'] <= 55)&(data['days_employed'] > 16920), 'days_employed'] = median_51_55

In [36]:
print(data[data['days_employed'] > 16920]['dob_years'].value_counts())

59    254
60    243
62    235
61    216
57    212
58    208
63    192
56    184
64    179
66    139
65    136
67    132
68     80
69     74
70     54
71     48
72     28
73      6
74      4
Name: dob_years, dtype: int64


In the columns printed above, we can see that we have replaced some of the anomalous values, leaving only anomalously high values for ages over 56 years old. We will leave these data unchanged, because original data is needed. It is difficult to pick up and guess what correct data should be there. Therefore, in order to finally correct this anomaly, it is necessary to request additional data.

Fill in the missing values in this column by also calculating the median over the already corrected values. The algorithm will be the same as above. We will use the get_median_years_employed () function written by to fill the intervals, for intervals from 56 years we will calculate only medians.

In [37]:
# calculate one of the new median values
median_update_46_50 = get_median_years_employed(46, 50)

In [38]:
# print out one of the values of the newly calculated medians 
print('Медиана стажа работы в категории от 46 до 50 лет:', median_update_46_50)

Медиана стажа работы в категории от 46 до 50 лет: 2411.5253843148307


In this example, we saw the main advantage of the median. Even in the presence of abnormally high values, both before and after their adjustment, the median values are the same. Therefore, to fill in the missing values, we will use the already calculated values. Passes will also be filled in by age category.

In [39]:
data.loc[(data['dob_years'] <= 25)&(data['days_employed'].isna()), 'days_employed'] = median_before_25
data.loc[(data['dob_years'] >= 26)&(data['dob_years'] <= 30)&(data['days_employed'].isna()), 'days_employed'] = median_26_30
data.loc[(data['dob_years'] >= 31)&(data['dob_years'] <= 35)&(data['days_employed'].isna()), 'days_employed'] = median_31_35
data.loc[(data['dob_years'] >= 36)&(data['dob_years'] <= 40)&(data['days_employed'].isna()), 'days_employed'] = median_36_40
data.loc[(data['dob_years'] >= 41)&(data['dob_years'] <= 45)&(data['days_employed'].isna()), 'days_employed'] = median_41_45
data.loc[(data['dob_years'] >= 46)&(data['dob_years'] <= 50)&(data['days_employed'].isna()), 'days_employed'] = median_46_50
data.loc[(data['dob_years'] >= 51)&(data['dob_years'] <= 55)&(data['days_employed'].isna()), 'days_employed'] = median_51_55
data.loc[(data['dob_years'] >= 56)&(data['dob_years'] <= 60)&(data['days_employed'].isna()), 'days_employed'] = median_56_60
data.loc[(data['dob_years'] >= 61)&(data['dob_years'] <= 65)&(data['days_employed'].isna()), 'days_employed'] = median_61_65
data.loc[(data['dob_years'] >= 66)&(data['dob_years'] <= 70)&(data['days_employed'].isna()), 'days_employed'] = median_66_70
data.loc[(data['dob_years'] >= 71)&(data['days_employed'].isna()), 'days_employed'] = median_upper_71

The missing values in the days_employed column have been filled in, but the anomalous values for age categories over 56 remain. The correctness of the specified data needs to be clarified, it is necessary to see the initial data for the correct correction of anomalies. In our analysis, these data will not be useful to us, but if in the future we need an analysis depending on the length of service, then the data for this is restored.

Now let's look at the total_income column to fill in the missing values. The table shows that all clients are broken down into categories of earnings, therefore, in order to more accurately indicate income at the place of passes, we also find out the median for each category. In this case, we will get less discrepancies than when calculating the average value.

In [40]:
# display the unique values of the income_type column
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64


So, we see categories of income from customers. Determine the median for each category in order to fill in the missing values with these values.

In [41]:
# define the median for four categories: employee - employee, companion - companion, pensioner - pensioner, 
# civil servant - civil_servant
employee = data.loc[data['income_type'] == 'сотрудник']['total_income'].median()
companion = data.loc[data['income_type'] == 'компаньон']['total_income'].median()
pensioner = data.loc[data['income_type'] == 'пенсионер']['total_income'].median()
civil_servant = data.loc[data['income_type'] == 'госслужащий']['total_income'].median()

# for entrepreneurs we will take the average value, since there are only two of them
businessmen = data.loc[data['income_type'] == 'предприниматель']['total_income'].mean()

In [42]:
# display the resulting median and average value
print('Медиана дохода по сотрудникам:', employee)
print('Медиана дохода по компаньонам:', companion)
print('Медиана дохода по пенсионерам:', pensioner)
print('Медиана дохода по госслужащим:', civil_servant)
print('Среднее значение дохода по предпринимателям:', businessmen)

Медиана дохода по сотрудникам: 142594.39684740017
Медиана дохода по компаньонам: 172357.95096577113
Медиана дохода по пенсионерам: 118514.48641164352
Медиана дохода по госслужащим: 150447.9352830068
Среднее значение дохода по предпринимателям: 499163.1449470857


In [43]:
# replace the missing values in the total_income column with medians by category
data.loc[(data['income_type'] == 'сотрудник')&(data['total_income'].isna()), 'total_income'] = employee
data.loc[(data['income_type'] == 'компаньон')&(data['total_income'].isna()), 'total_income'] = companion
data.loc[(data['income_type'] == 'пенсионер')&(data['total_income'].isna()), 'total_income'] = pensioner
data.loc[(data['income_type'] == 'госслужащий')&(data['total_income'].isna()), 'total_income'] = civil_servant
data.loc[(data['income_type'] == 'предприниматель')&(data['total_income'].isna()), 'total_income'] = businessmen

Check if all the missing values in the total_income column are filled

In [44]:
# checking for missing values
print(data.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Now all the missing values are eliminated, we can proceed to the next step.

**Conclusions**

The missing values were in two columns:
- days_employed
- total_income

With the total_income column, there was no difficulty in filling in the missing values, all customers could be categorized according to the type of earnings (income_type). 5 types of earnings were identified, for four of them we calculated the value of the median and filled in the missing values with these values. But for one category, entrepreneur, we calculated the average value, since there were 2 such clients, so the median value is not informative here.

It took longer to deal with the days_employed column, since this column had a lot of anomalous values (more than 900 years of work experience!), Some of these anomalous values were corrected by replacing the anomaly with a median calculated depending on the age category. We automated the calculation of medians by writing a function. We did this in order not to duplicate the code. After that, they were replaced by medians and the remaining gaps in the data. But there are a lot og questions to this column, as for 2624 rows, almost 12% of the data, the values also remain anomalous. In our analysis, this column will not be useful, but for further analysis it is necessary to find out the reason why incorrect data was entered: human error, problems with saving data, incorrect data conversion, etc.

### Replacing the data type

Let's replace the data type in two columns. Let's display the information on the screen again.

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Columns days_employed and total_income have data type float64, replace the real data type with integer.

In [46]:
# replace the data type in the days_employed column
data['days_employed'] = data['days_employed'].astype('int')

In [47]:
# replace the data type in the total_income column
data['total_income'] = data['total_income'].astype('int')

In [48]:
# display information on data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [49]:
# output 5 rows of data to make sure that type changes have occurred
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование
4,0,3682,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу


**Conclusion**

As we can see above, the data in the days_employed and total_income columns has become integer.

Now, for further convenience of working with the table, we will remove the existing duplicates.

### Duplicate Handling

During the initial analysis of the table, it was revealed that there may be obvious duplicates in our dataframe. Display the number of duplicates again.

In [50]:
# displaying the number of implicit duplicates
print(data.duplicated().sum())

72


In total, there were 72 rows of obvious duplicates, which is about 0.33%. This value is small, so removing obvious duplicates should not affect the final results in any way. Therefore, we will remove explicit duplicates with the removal of old indices and the formation of new ones.

In [51]:
# removal of explicit duplicates (with the removal of old indexes and the formation of new ones)
data = data.drop_duplicates().reset_index(drop=True)

In [52]:
# once again display the number of explicit duplicates
print(data.duplicated().sum())

0


**Conclusion**

All explicit duplicates were removed from the dataframe, old indexes were removed and new ones were formed.

Now the dataframe is ready for analysis: all missing values were filled in, data were converted to the same register, data types were changed where necessary, explicit duplicates were removed, all anomalies in values that could be corrected had been corrected.

### Lemmatization

In the last column of the list, we see that, in general, the purposes of the loan correspond to several categories, but due to the fact that these words are written differently: there are declensions, additional words, it is very difficult to reduce all this data to several categories. Let's try to lemmatize this data.

In [53]:
# for lemmatization, import the module 
from pymystem3 import Mystem
m = Mystem()

In [54]:
# import Counter from collection module to count lemmas
from collections import Counter

Let's analyze the number of lemmas in the purpose column

In [55]:
# let's bring all the goals in the purpose column to their lemmas and display this data
data_join = ','.join(data['purpose'])
lemma = m.lemmatize(data_join)
print(lemma)

['покупка', ' ', 'жилье', ',', 'приобретение', ' ', 'автомобиль', ',', 'покупка', ' ', 'жилье', ',', 'дополнительный', ' ', 'образование', ',', 'сыграть', ' ', 'свадьба', ',', 'покупка', ' ', 'жилье', ',', 'операция', ' ', 'с', ' ', 'жилье', ',', 'образование', ',', 'на', ' ', 'проведение', ' ', 'свадьба', ',', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ',', 'покупка', ' ', 'недвижимость', ',', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ',', 'сыграть', ' ', 'свадьба', ',', 'приобретение', ' ', 'автомобиль', ',', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ',', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ',', 'недвижимость', ',', 'строительство', ' ', 'недвижимость', ',', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ',', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ',', 'недвижимость', ',', 'приобретение', ' ', 'автомобиль', ',', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ',', 'сыграть', ' ', 'свадьба', ',', 'опера

In [56]:
# count the number of frequently repeated lemmas in order to find base words for various names in the purpose column
print(Counter(lemma))

Counter({' ': 33568, ',': 21452, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2323, 'свой': 2230, 'на': 2221, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 767, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


By counting the number of lemmas, we can categorize the data in the purpose column AND highlight the main areas for which a loan is taken:
- real estate (real estate, housing, residential)
- car (car)
- wedding (wedding)
- education (education)

Thus, we can write a function to define categories for each loan purpose. Will be implemented below in the paragraph data categorization.

**Conclusion**

Lemmatization made it very easy to combine similar data to categorize it for further analysis.
We don't need to write complicated loops to iterate over all possible values. Instead, we have reduced all the words to their lemmas and identified the most frequently occurring data. Next, we categorize the data based on the information received.

### Data categorization

We need to categorize the data according to four types of data, because 4 questions need to be answered:
- categorize the children column
- categorize the family_status column
- categorize the total_income column
- categorize the purpose column

During the categorization of data, we will add columns with categories to the dataframe. In order to be able to use these categories in the future to answer the questions posed.

For ease of analysis, we will divide all clients into the following categories, depending on the number of children. The generally accepted classification is as follows:
- clients without children (number of children 0)
- clients with children (number of children from 1)

Let's create a function to distribute the number of children by category.

In [57]:
# function to categorize by the number of children
def get_category_children(row):
    '''
    
    Returns the category based on the number of children
    
    There are 2 categories: clients without children (children == 0), clients with children (everyone else).
    param: row dataframe column data['children']
    '''
    children = row['children']
    if children == 0:
        return 'клиенты без детей'
    else:
        return 'клиенты с детьми'

In [58]:
# apply this function and create a new column
data['children_category'] = data.apply(get_category_children, axis=1)

In [59]:
# check that the category we need has been created in the column
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,клиенты с детьми
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,клиенты с детьми
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,клиенты без детей
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,клиенты с детьми
4,0,3682,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,клиенты без детей


For the family_status column, in my opinion, the required data categorization is already present in the source table. Therefore, we will use the following categories:
- married
- unmarried
- civil marriage
- divorced
- widower


Let's categorize the data for the total_income column.
Income can be divided into three categories:
- high income
- average income
- low income

For a more accurate classification of clients depending on the level of income, we will carry out a two-level formation of the interval for clients with an average income level. We use one of the properties of the median for this: half of the set of elements is not less than the median, and the second half is not more. First we find the median_general for the entire set of values in total_income, and then we perform this operation twice more: to find the median_lower of the set of elements that are before the first median_general, and determine the median_upper of the set of elements that are after the first median_general. Thus, in my opinion, we will get a more reliable distribution of customer income by category.

In [60]:
# define median_general 
median_general = int(data['total_income'].median())
print('median_general:', median_general)

median_general: 142594


In [61]:
# define median_lower
median_lower = int(data.loc[data['total_income'] < median_general]['total_income'].median())
print('median_lower:', median_lower)

median_lower: 103460


In [62]:
# define median_second_volume 
median_upper = int(data.loc[data['total_income'] > median_general]['total_income'].median())
print('median_upper', median_upper)

median_upper 196593


Thus, we have obtained the necessary values and it is possible to categorize the data into the following categories:
- high level of income (total_income >= median_upper)
- average income level (median_lower < total_income < median_upper)
- low income level (total_income <= median_lower)

Let's write a function for distributing customers by category.

In [63]:
# function to categorize by income level
def get_category_total_income(row, median_lower, median_upper):
    '''
    Returns a category based on income level
     
    There are 3 categories: high income (total_income >= median_upper), 
    average income (median_first_volume < total_income < median_upper),
    low income (total_income < median_first_volume)
    param: row is a row of dataframe data['total_income']
    param: median_upper
    param: median_lower
    
    '''    
    total_income = row['total_income']
    if total_income >= median_upper:
        return 'высокий уровень дохода'
    if median_lower < total_income < median_upper:
        return 'средний уровень дохода'
    if total_income <= median_lower:
        return 'низкий уровень дохода'

In [64]:
# apply this function and create a new column
data['total_income_category'] = data.apply(get_category_total_income, args=(median_lower, median_upper), axis=1)

In [65]:
# check that the category we need has been created in the column
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,клиенты с детьми,высокий уровень дохода
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,клиенты с детьми,средний уровень дохода
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,клиенты без детей,средний уровень дохода
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,клиенты с детьми,высокий уровень дохода
4,0,3682,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,клиенты без детей,средний уровень дохода


And finally, we will select categories for the purpose column. By counting the number of lemmas, we can categorize the data in the purpose column:

- real estate (real estate, housing, residential)
- car (car)
- wedding (wedding)
- education (education)

Thus, we can write a function to determine the categories for each purpose of obtaining a loan.

In [66]:
# write a function to iterate through all lems and define categories
def get_category_purpose(row):
    '''
    Returns the category depending on the purpose of the loan
    
    param: row row from data, column purpose
    '''
    lem_purpose = m.lemmatize(row['purpose'])
    if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose) or ('жилой' in lem_purpose):
        return 'недвижимость'
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if 'свадьба' in lem_purpose:
        return 'свадьба'
    else:
        return 'образование'

In [67]:
# apply this function and create a new column
data['purpose_category'] = data.apply(get_category_purpose, axis=1)

In [68]:
# check that the category we need has been created in the column
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category,total_income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,клиенты с детьми,высокий уровень дохода,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,клиенты с детьми,средний уровень дохода,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,клиенты без детей,средний уровень дохода,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,клиенты с детьми,высокий уровень дохода,образование
4,0,3682,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,клиенты без детей,средний уровень дохода,свадьба


In [69]:
# output a dictionary by category children_category
print(data['children_category'].value_counts())

клиенты без детей    14143
клиенты с детьми      7310
Name: children_category, dtype: int64


In [70]:
# display a dictionary by category family_status
print(data['family_status'].value_counts())

женат / замужем          12339
гражданский брак          4150
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


In [71]:
# display a dictionary by category total_income_category
print(data['total_income_category'].value_counts())

средний уровень дохода    11260
высокий уровень дохода     5306
низкий уровень дохода      4887
Name: total_income_category, dtype: int64


In [72]:
# display a dictionary vy category purpose_category
print(data['purpose_category'].value_counts())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2323
Name: purpose_category, dtype: int64


**Conclusion**

The data categorization was carried out for four columns, so that in the next section it was possible to answer the questions posed.
The following category dictionaries have been identified:
1. by children column:
    - clients without children
    - clients with children
2. by the family_status column:
    - married
    - unmarried
    - civil marriage
    - divorced
    - widow/widower
3. by the total_income column:
    - high income
    - average income
    - low income
4. by purpose column:
    - real estate
    - car
    - education
    - wedding
    
Now the data is complete. We will analyze further and answer the questions posed

## Step 3: Answer the questions

- Is there a relationship between having children and repaying a loan on time?

To answer the question of whether there is a relationship between having children and repaying a loan on time, it is necessary to create a table by categories of clients, calculate the frequency of occurrence of debts outstanding on time using these parameters. To do this, we will need to calculate this ratio by dividing the number of outstanding loans by the total number of values in the sample. The higher this value, the more likely that clients from this category will not repay the loan on time.

In [73]:
# we will create a pivot table in order to answer the question and calculate the share of debts
pivot_table_children = data.pivot_table(index='children_category', columns='debt', values='children', aggfunc='count')
pivot_table_children['share'] = pivot_table_children[1] / (pivot_table_children[1] + pivot_table_children[0])

In [74]:
# display the final table
display(pivot_table_children.sort_values('share', ascending=False))

debt,0,1,share
children_category,,,
клиенты с детьми,6636,674,0.092202
клиенты без детей,13076,1067,0.075444


**Conclusion**

In the resulting table 0 means no debts, 1 means there are debts, share is the share of clients who have debts.

The resulting table shows that there is a relationship between having children and repaying the loan on time.
So, customers who do not have children are the least likely to have problems with loan repayment on time. But clients with children most often pay off the loan on time.

- Is there a relationship between marital status and loan repayment on time?

To answer the question of whether there is a relationship between marital status and loan repayment on time, it is necessary to create a table by customer categories, calculate the frequency of occurrence of debts outstanding on time using these parameters. To do this, we will need to calculate this ratio by dividing the number of loans outstanding on time by the total number of values in the sample. Than the higher is this value, the more likely that clients from this category will not repay the loan on time.

In [75]:
# we will create a pivot table in order to answer the question and calculate the share of debts
pivot_table_family_status = data.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
pivot_table_family_status['share'] = pivot_table_family_status[1] / (pivot_table_family_status[1] + pivot_table_family_status[0])

In [76]:
# display final table
display(pivot_table_family_status.sort_values('share', ascending=False))

debt,0,1,share
family_status,,,
не женат / не замужем,2536,274,0.097509
гражданский брак,3762,388,0.093494
женат / замужем,11408,931,0.075452
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693


**Conclusion**

According to the obtained table, it can be seen that there is a relationship between marital status and loan repayment on time. So, the least likely to fail to pay loans on time are clients who belong to the category of a widower / widow, a little less responsible in paying debts - divorced, and married. But those who are not married most often have delays in paying a loan, as well as often have delays in loan payments and customers who are in a civil marriage.

- Is there a relationship between income level and loan repayment on time?

To answer the question of whether there is a dependence between the level of income and repayment of the loan on time, it is necessary to form a table by customer categories, calculate the occurrence rate of outstanding debts using these parameters. To do this, we will need to calculate this ratio by dividing the number of overdue loans by the total number of values in the sample. Than the higher is this value, the more likely that clients from this category will not repay the loan on time.

In [77]:
# determine the number of outstanding loans, as well as the total number of loans by category
data_grouped_total_income_category = data.groupby('total_income_category').agg({'debt': ['count', 'sum']})

In [78]:
# calculate the occurrence rate of outstanding debts, use the try-except construct, because can occur, for example, division by 0
try:
    data_grouped_total_income_category['friquency_debt_total_income'] = data_grouped_total_income_category['debt']['sum'] / data_grouped_total_income_category['debt']['count']
except ValueError:
    data_grouped_total_income_category['friquency_debt_total_income'] = 0

In [79]:
# display data
display(data_grouped_total_income_category.sort_values('friquency_debt_total_income', ascending=False))  

debt      friquency_debt_total_income
                        count  sum                            
total_income_category                                         
средний уровень дохода  11260  980                    0.087034
низкий уровень дохода    4887  384                    0.078576
высокий уровень дохода   5306  377                    0.071052

**Conclusion**

Compared to previous conclusions, it is not possible to unambiguously say in this conclusion whether the level of income affects the presence of outstanding loans. The values do not differ much from each other, but even these data show that high- and low-income clients are less likely to default on loans than middle-income clients.

- How do different purposes of a loan affect its repayment on time?

To answer the question of whether there is a relationship between the goals of obtaining a loan and repaying the loan on time, it is necessary to create a table by customer categories, calculate the occurrence rate of outstanding debts using these parameters. To do this, we will need to calculate this ratio by dividing the number of overdue loans by the total number of values in the sample. Than the higher is this value, the more likely that clients from this category will not repay the loan on time.

In [80]:
# determine the number of outstanding loans, as well as the total number of loans by category
data_grouped_purpose_category = data.groupby('purpose_category').agg({'debt': ['count', 'sum']})

In [81]:
# calculate the occurrence rate of outstanding debts, use the try-except construct, because can occur, for example, division by 0
try:
    data_grouped_purpose_category['friquency_debt_purpose'] = data_grouped_purpose_category['debt']['sum'] / data_grouped_purpose_category['debt']['count']
except ValueError:
    data_grouped_purpose_category['friquency_debt_purpose'] = 0

In [82]:
# display data
display(data_grouped_purpose_category.sort_values('friquency_debt_purpose', ascending=False))  

debt      friquency_debt_purpose
                  count  sum                       
purpose_category                                   
автомобиль         4306  403               0.093590
образование        4013  370               0.092200
свадьба            2323  186               0.080069
недвижимость      10811  782               0.072334

**Conclusion**

Based on the obtained data, it can be concluded that there is a relationship between the goals of obtaining a loan and the presence of delays in payment. So people whose purpose of obtaining a loan is real estate and a wedding are more responsible in paying off a loan. Less responsible for repayment of loans and, as a result, those who have the most delays in payments are customers whose purpose of obtaining a car and education.

## Step 4. General conclusion

During the course of the project, the following was done:
1. The table was processed: duplicates were removed, missing values were identified and filled in, anomalies in values were identified and corrected, all values were reduced to the same register, some data types were also changed: real to integer (in order to avoid problems due to rounding data).
2. Work was carried out on the lemmatization of a part of the data in order to make it possible to categorize some of the data.
3. Categories were identified depending on the number of children, marital status, income level, goals for obtaining a loan.
4. Based on the categorization, we were able to answer the questions posed:
    - is there a relationship between having children and repaying a loan on time: yes, there is a relationship. Most often, clients with children have problems with loan repayment, and childless clients often repay the loan on time;
    - is there a relationship between marital status and loan repayment on time: yes, there is a relationship. Most often, clients who are not married or who are in a civil marriage do not repay loans on time. People who are married or divorced are more responsible for repaying a loan. The clients from the widower/widow category turned out to be the most disciplined;
    - is there a relationship between the level of income and repayment of the loan on time: according to the obtained data, it is impossible to give an unambiguous answer to this question, since the obtained values differ slightly from each other. But everything can be distinguished that clients with high and low income levels repay loans responsibly;
    - how different goals of the loan affect its repayment on time: we can definitely say that there is a dependence. Clients are less likely to default on loans if the purpose of the loan is real estate and weddings, and often have problems repaying loans if the purpose is education and a car.
    
The answers to the questions posed have been given. Research completed.